# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json as json


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Loading Output CSV 

file_one="../Output/all_cities_df.csv"
file_df=pd.read_csv(file_one)
file_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,turukhansk,65.8167,87.9833,-16.71,85,2,6.85,RU,1615434716
1,kodiak,57.7900,-152.4072,33.80,47,1,23.02,US,1615434716
2,yar-sale,66.8333,70.8333,-28.25,83,0,6.98,RU,1615434717
3,ginda,30.6945,78.4932,68.00,56,0,3.44,IN,1615434717
4,mount isa,-20.7333,139.5000,89.60,45,75,6.91,AU,1615434717


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configuring Google Maps

gmaps.configure(api_key=g_key)

In [4]:
# Storing variables:

locations=file_df[['Lat','Lng']] # Latitude and Longitude as 'Locations'
weight=file_df['Humidity'].astype(float) # Humidity as 'Weight'

In [5]:
# Plotting a Heatmap: Humidity 

# Plot
fig=gmaps.figure() 

# Creating a heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight,  
                                 dissipating=False, max_intensity=300,
                                 point_radius=3)

# Adding the heatmap layer
fig.add_layer(heat_layer)

# Displaying the plot
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Questionary for the client:

# Asking the traveller for the desired temperature to have in the city to visit: min_temp
min_temp=float(input("What is the minimum temperature you'd like to have for your trip (in fahrenheit)? "))

# Asking the traveller for the desired temperature to have in the city to visit: max_temp
max_temp=float(input("What is the maximum temperature you'd like to have for your trip (in Fahrenheit)? "))

# Asking the traveller for the desired humidity to have in the city to visit: max_hum
max_hum=float(input("What is the maximum humidity you'd like to have for your trip? "))

# Asking the traveller for the desired cloudiness to have in the city to visit: max_cloud
max_cloud=float(input("What is the maximum cloudiness you'd like to have for your trip? "))


What is the minimum temperature you'd like to have for your trip (in fahrenheit)? 1
What is the maximum temperature you'd like to have for your trip (in Fahrenheit)? 50
What is the maximum humidity you'd like to have for your trip? 50
What is the maximum cloudiness you'd like to have for your trip? 28


In [7]:
# Passing the clients answers to variables to get all cities that match those requirements

cities_choice = file_df.loc[(file_df["Max Temp"] >= min_temp) & (file_df["Max Temp"] <= max_temp) & \
                            (file_df["Humidity"] <= max_hum) & (file_df["Cloudiness"] <= max_cloud)]

                            
print(f"There are {len(cities_choice)} cities around the world that match the required specifications for the clients' needs.")
cities_choice.head(3)

There are 8 cities around the world that match the required specifications for the clients' needs.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,kodiak,57.7900,-152.4072,33.8,47,1,23.02,US,1615434716
10,chase,50.8165,-119.6857,39.2,35,0,9.22,CA,1615434719
47,prince george,53.9166,-122.7530,23.0,39,1,3.44,CA,1615434732


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Adding a "Hotel Name" & "Rating" column to the DataFrame

cities_choice['Hotel Name'] = ""
cities_choice['Hotel Rating'] = ""
cities_choice.head(3)

<ipython-input-8-2699521ae2ad>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities_choice['Hotel Name'] = ""
<ipython-input-8-2699521ae2ad>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities_choice['Hotel Rating'] = ""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Rating
1,kodiak,57.7900,-152.4072,33.8,47,1,23.02,US,1615434716,,
10,chase,50.8165,-119.6857,39.2,35,0,9.22,CA,1615434719,,
47,prince george,53.9166,-122.7530,23.0,39,1,3.44,CA,1615434732,,


In [9]:
print(f"There are {len(cities_choice)} cities to choose from.")

There are 8 cities to choose from.


In [10]:
# For Loop to get the data into 'Hotel Name' and 'Hotel Rating' columns

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Base URL to work with
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# Setting variables
hotel_info = [info_box_template.format(**row) for index, row in cities_choice.iterrows()]
locations = cities_choice[["Lat", "Lng"]].astype(float)


# Setting parameters
params={
    "radius":5000,
    "type": "hotel",
    "key": g_key,
}

# Iterate through 
for index, row in cities_choice.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        cities_choice.loc[index, "Hotel Name"] = name_address["results"][1]["name"]
        cities_choice.loc[index, "Hotel Rating"] = name_address["results"][1]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
print(json.dumps(name_address, indent=4, sort_keys=True))

C:\Users\fernando\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}


In [11]:
# Printing out the table with the results in columns 'Hotel Name' & 'Hotel Rating'

cities_choice.head(3)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Rating
1,kodiak,57.7900,-152.4072,33.8,47,1,23.02,US,1615434716,US Coast Guard,4.7
10,chase,50.8165,-119.6857,39.2,35,0,9.22,CA,1615434719,Fas Gas Plus,4
47,prince george,53.9166,-122.7530,23.0,39,1,3.44,CA,1615434732,Canadas Best Value Inn Prince George,3.9


In [12]:
# Passing the name of data frame to 'hotel_df'

hotel_df=cities_choice
hotel_df.head(3)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Rating
1,kodiak,57.7900,-152.4072,33.8,47,1,23.02,US,1615434716,US Coast Guard,4.7
10,chase,50.8165,-119.6857,39.2,35,0,9.22,CA,1615434719,Fas Gas Plus,4
47,prince george,53.9166,-122.7530,23.0,39,1,3.44,CA,1615434732,Canadas Best Value Inn Prince George,3.9


In [20]:
# Ploting markers on top of the heatmap

marker_locations = hotel_df[['Lat', 'Lng']]
markers = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig=gmaps.figure()
fig.add_layer(markers)

#fig.savefig("../Images/hoteldf_marker_heatmapA.png", dpi=300, bbox_inches='tight')

fig

Figure(layout=FigureLayout(height='420px'))